In [1]:
import cv2
import time
import numpy as np
import HandTrackingModule as htm
import os

In [2]:
# Initialize camera
cap = cv2.VideoCapture(0)

# Set camera width and height
w_cam = 1280
h_cam = 720

cap.set(3, w_cam)  # sets width
cap.set(4, h_cam)  # sets height

# Load the image to drag and drop
image_path = "images/coke.jpeg"
image = cv2.imread(image_path)

# Resize the image if necessary
image_height, image_width = image.shape[:2]
if image_height > h_cam or image_width > w_cam:
    scaling_factor = min(h_cam / image_height, w_cam / image_width)
    image = cv2.resize(image, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)
    image_height, image_width = image.shape[:2]

# Initialize variables
pTime = 0
detector = htm.handDetector(detectionCon=0.50)
dragging = False
start_x, start_y = 0, 0
offset_x, offset_y = 0, 0

while True:
    success, img = cap.read()
    if not success:
        break

    img = cv2.flip(img, 1)  # Flip the image horizontally

    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)

    if len(lmList) != 0:
        # Get the tip positions of the thumb and index finger
        thumb_tip = lmList[4]
        index_tip = lmList[8]

        # Calculate the midpoint between the thumb and index finger
        cx, cy = (thumb_tip[1] + index_tip[1]) // 2, (thumb_tip[2] + index_tip[2]) // 2

        # Check if the fingers are close enough to start dragging
        distance = np.hypot(index_tip[1] - thumb_tip[1], index_tip[2] - thumb_tip[2])
        if distance < 40:  # Threshold for "pinching"
            if not dragging:
                # Calculate offset
                offset_x, offset_y = cx - start_x, cy - start_y
                dragging = True
            else:
                # Update the image position
                start_x, start_y = cx - offset_x, cy - offset_y
        else:
            dragging = False

    if dragging:
        # Calculate the new position of the image
        x1, y1 = max(0, min(w_cam - image_width, start_x)), max(0, min(h_cam - image_height, start_y))

        # Create a transparent overlay
        overlay = img.copy()
        overlay[y1:y1 + image_height, x1:x1 + image_width] = image

        # Add the overlay
        img = cv2.addWeighted(overlay, 0.5, img, 0.5, 0)
    else:
        # Ensure the coordinates are within bounds
        x1, y1 = max(0, min(w_cam - image_width, start_x)), max(0, min(h_cam - image_height, start_y))
        
        # Display the image in the last known position
        img[y1:y1 + image_height, x1:x1 + image_width] = image

    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime

    cv2.putText(img, f'FPS={int(fps)}', (50, h_cam - 90), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 3)
    cv2.imshow("DragDrop", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\Users\shory\anaconda3\envs\myenv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
